# Exercise

A toy manufacturing organization manufactures two types of toys A and B. Both the toys are sold at 25€ and 20€ respectively. There are 2000 resource units available every day from which the toy A requires 20 units while toy B requires 12 units. Both of these toys require a production time of 5 minutes. Total working hours are 9 hours a day. What should be the manufacturing quantity for each of the pipes to maximize the profits ?

In [65]:
from pulp import *

prob = LpProblem("Toys_Problem", LpMaximize)

#Crete problem variables
x = LpVariable("Toy A", 0, None, LpInteger)
y = LpVariable("Toy B", 0, None, LpInteger)

#The objective function is added
prob += 25*x + 20*y, "Total profit; to be maximized"

#The two constraints are entered
prob += 20*x + 12*y <=2000, "Constraint of the resource"
prob += 5*x + 5*y <= 540, "Constraint of the time"

prob.writeLP("Toys_Problem.lp")

prob.solve()

1

In [66]:
#The status of the solution
print("Status:", LpStatus[prob.status])

Status: Optimal


In [67]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

Toy_A = 88.0
Toy_B = 20.0


In [68]:
#The optimised objective function value
print("Total Profit = ", value(prob.objective))

Total Profit =  2600.0


Our aim is to help Mr Doe to choose some sites to visit during these two days. You can use some linear programming models or another approach to do this. Use a python code to answer the following questions :

1. It is assumed that Mr. Doe gives equal importance to each tourist site, and he wants to visit the maximum number of sites. Which list(s) of places could you recommend to him ? This solution will be called ListVisit 1.

In [70]:
from numpy import matrixlib as npmat
import pandas as pd
import numpy as np
places = ["TE", "ML", "AT", "MO", "JT", "CA", "CP", "CN", "BS", "SC", "PC", "TM", "AC"]

places_names = {"TE": "La Tour Eiffel", "ML":"Le Musée du louvre", "AT":"l’Arc de triomphe", 
                "MO":"le Musée d’Orsay", "JT":"le Jardin des tuileries", "CA":"les Catacombes", 
                "CP":"le Centre Pompidou", "CN":"la Cathédrale Notre Dame de Paris", "BS":"la Basilique du Sacré-Coeur",
                "SC":"la Sainte Chapelle", "PC":"La Place de la Concorde", "TM":"la Tour Montparnasse", "AC":"l’Avenue des Champs-Elysées"}

duration = [4.5, 3, 1, 2, 1.5, 2, 2.5, 2, 2, 1.5, 0.75, 2, 1.5]
price = [15.5, 12, 9.5, 11, 0, 10, 10, 5, 8, 8.5, 0, 15, 0]
rating = [5, 4, 3, 2, 3, 4, 1, 5, 4, 1, 3, 2, 5]

In [71]:
List_Visit_1=[]

def ListVisit1():
    prob = LpProblem("How_to_visit_Paris", LpMaximize)
    
    rec_place = []
    for place in places:
        rec_place.append(LpVariable(place, 0, 1, LpInteger))

    #Objective function - ListVisit1
    number_of_sites = 0
    for place in rec_place:
        number_of_sites += place

    prob += number_of_sites, "Objective Function"

    #Basic constraints of time and price
    index = 0
    places_total_price = 0
    places_total_duration = 0

    for r in rec_place:
        places_total_price += r * price[index]
        places_total_duration += r * duration[index]
        index += 1

    prob += places_total_price <= 65, "Constraint of the price"
    prob += places_total_duration <= 12, "Constraint of the duration"

    prob.writeLP("How_to_visit_Paris.lp")
    prob.solve() 
    
    for v in prob.variables():
        if v.varValue == 1.0:
            print(v.name, "-", places_names[v.name])
            List_Visit_1.append(v.name)
    print("Number of places in the ListVisit - ", value(prob.objective))
            
ListVisit1()
List_Visit_1

AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
BS - la Basilique du Sacré-Coeur
CA - les Catacombes
CN - la Cathédrale Notre Dame de Paris
PC - La Place de la Concorde
SC - la Sainte Chapelle
Number of places in the ListVisit -  7.0


['AC', 'AT', 'BS', 'CA', 'CN', 'PC', 'SC']

Actually, Mr. Doe has some preferences among these tourist sites and he expresses them as follows :

• Preference 1 : If two sites are geographically very close (within a radius of 1 km of walking), he will prefer to visit these two sites instead of visiting only one.

• Preference 2 : He absolutely wants to visit the Eiffel Tower (TE) and Catacombes (CA).

• Preference 3 : If he visits Notre Dame Cathedral (CN) then he will not visit the Sainte Chapelle (SC).

• Preference 4 : He absolutely wants to visit Tour Montparnasse (TM).

• Preference 5 : If he visits the Louvre (ML) Museum then he must visit the Pompidou Center (CP).

In [72]:
def how_to_visit_Paris(p1, p2, p3, p4, p5):
    rank=[]
    prob1 = LpProblem("How_to_visit_Paris", LpMaximize)
    
    rec_place = []
    for place in places:
        rec_place.append(LpVariable(place, 0, 1, LpInteger))

    #Objective function
    number_of_sites = 0
    for place in rec_place:
        number_of_sites += place

    prob1 += number_of_sites, "Objective Function"

    #Basic constraints of time and price
    index = 0
    places_total_price = 0
    places_total_duration = 0

    for r in rec_place:
        places_total_price += r * price[index]
        places_total_duration += r * duration[index]
        index += 1

    prob1 += places_total_price <= 65, "Constraint of the price"
    prob1 += places_total_duration <= 12, "Constraint of the duration"
        
    """Preference constraint"""
    if p1 == 1:
        prob1 += rec_place[3] == rec_place[4]  #(MO,JT)
        prob1 += rec_place[6] == rec_place[7]  #(CN,CP)
        prob1 += rec_place[1] == rec_place[6] == rec_place[7] == rec_place[9]  #(SC,ML,CP,CN)
    if p2 == 1:
        prob1 += rec_place[0] + rec_place[5] == 2
    if p3 == 1:
        prob1 += rec_place[7] + rec_place[9] <= 1
    if p4 == 1:
        prob1 += rec_place[11] == 1
    if p5 == 1:
        prob1 += rec_place[1] == rec_place[6]

    prob1.writeLP("How_to_visit_Paris.lp")
    prob1.solve() 
    
    for v in prob1.variables():
        if v.varValue == 1.0:
            print(v.name, "-", places_names[v.name])
            rank.append(v.name)
        
    print("Number of places in the ListVisit - ", value(prob1.objective))
    print()
    print("Are the obtained lists different from the solution ListVisit1 ?")
    if rank == List_Visit_1:
        print("Obtained list is the same as ListVisit1.")
    else:
        print("Obtained list is different from the ListVisit1.")

(a) For each of the five preferences above, suggest to Mr. Doe, one or more lists of tourist sites to visit. Are the obtained lists different from the solution ListVisit 1 ? To answer this last question, you can implement a python function returning True (respectively False) if two lists are identical (respectively different).

In [73]:
print("List Visit with Preference 1:")
how_to_visit_Paris(1, 0, 0, 0, 0)

List Visit with Preference 1:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
JT - le Jardin des tuileries
MO - le Musée d’Orsay
PC - La Place de la Concorde
TM - la Tour Montparnasse
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


In [74]:
print("List Visit with Preference 2:")
how_to_visit_Paris(0, 1, 0, 0, 0)

List Visit with Preference 2:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
PC - La Place de la Concorde
SC - la Sainte Chapelle
TE - La Tour Eiffel
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


In [75]:
print("List Visit with Preference 3:")
how_to_visit_Paris(0, 0, 1, 0, 0)

List Visit with Preference 3:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
BS - la Basilique du Sacré-Coeur
CA - les Catacombes
MO - le Musée d’Orsay
PC - La Place de la Concorde
SC - la Sainte Chapelle
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


In [76]:
print("List Visit with Preference 4:")
how_to_visit_Paris(0, 0, 0, 1, 0)

List Visit with Preference 4:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
BS - la Basilique du Sacré-Coeur
CN - la Cathédrale Notre Dame de Paris
PC - La Place de la Concorde
SC - la Sainte Chapelle
TM - la Tour Montparnasse
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


In [77]:
print("List Visit with Preference 5:")
how_to_visit_Paris(0, 0, 0, 0, 1)

List Visit with Preference 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
BS - la Basilique du Sacré-Coeur
CN - la Cathédrale Notre Dame de Paris
MO - le Musée d’Orsay
PC - La Place de la Concorde
SC - la Sainte Chapelle
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(b) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 2, which list(s) would you recommend to him ?

In [78]:
print("List Visit with Preferences 1 and 2:")
how_to_visit_Paris(1, 1, 0, 0, 0)

List Visit with Preferences 1 and 2:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
PC - La Place de la Concorde
TE - La Tour Eiffel
TM - la Tour Montparnasse
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(c) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 3, which list(s) would you recommend to him ?

In [79]:
print("List Visit with Preferences 1 and 3:")
how_to_visit_Paris(1, 0, 1, 0, 0)

List Visit with Preferences 1 and 3:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
JT - le Jardin des tuileries
MO - le Musée d’Orsay
PC - La Place de la Concorde
TM - la Tour Montparnasse
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(d) If Mr. Doe wishes, at the same time, to take into account Preference 1 and Preference 4, which list(s) would you recommend to him ?

In [80]:
print("List Visit with Preferences 1 and 4:")
how_to_visit_Paris(1, 0, 0, 1, 0)

List Visit with Preferences 1 and 4:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
JT - le Jardin des tuileries
MO - le Musée d’Orsay
PC - La Place de la Concorde
TM - la Tour Montparnasse
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(e) If Mr. Doe wishes, at the same time, to take into account Preference 2 and Preference 5, which list(s) would you recommend to him ?

In [81]:
print("List Visit with Preferences 2 and 5:")
how_to_visit_Paris(0, 1, 0, 0, 1)

List Visit with Preferences 2 and 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
JT - le Jardin des tuileries
PC - La Place de la Concorde
TE - La Tour Eiffel
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(f) If Mr. Doe wishes, at the same time, to take into account Preference 3 and Preference 4, which list(s) would you recommend to him ?

In [82]:
print("List Visit with Preferences 3 and 4:")
how_to_visit_Paris(0, 0, 1, 1, 0)

List Visit with Preferences 3 and 4:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
BS - la Basilique du Sacré-Coeur
CA - les Catacombes
PC - La Place de la Concorde
SC - la Sainte Chapelle
TM - la Tour Montparnasse
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(g) If Mr. Doe wishes, at the same time, to take into account Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [83]:
print("List Visit with Preferences 4 and 5:")
how_to_visit_Paris(0, 0, 0, 1, 1)

List Visit with Preferences 4 and 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CN - la Cathédrale Notre Dame de Paris
MO - le Musée d’Orsay
PC - La Place de la Concorde
SC - la Sainte Chapelle
TM - la Tour Montparnasse
Number of places in the ListVisit -  7.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(h) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2 and Preference 4, which list(s) would you recommend to him ?

In [84]:
print("List Visit with Preferences 1, 2 and 4:")
how_to_visit_Paris(1, 1, 0, 1, 0)

List Visit with Preferences 1, 2 and 4:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
PC - La Place de la Concorde
TE - La Tour Eiffel
TM - la Tour Montparnasse
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(i) If Mr. Doe wishes, at the same time, to take into account Preference 2, Preference 3 and Preference 5, which list(s) would you recommend to him ?

In [85]:
print("List Visit with Preferences 2, 3 and 5:")
how_to_visit_Paris(0, 1, 1, 0, 1)

List Visit with Preferences 2, 3 and 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
JT - le Jardin des tuileries
PC - La Place de la Concorde
TE - La Tour Eiffel
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(j) If Mr. Doe wishes, at the same time, to take into account Preference 2, Preference 3, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [86]:
print("List Visit with Preferences 2, 3, 4 and 5:")
how_to_visit_Paris(0, 1, 1, 1, 1)

List Visit with Preferences 2, 3, 4 and 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
PC - La Place de la Concorde
TE - La Tour Eiffel
TM - la Tour Montparnasse
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(k) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2, Preference 4 and Preference 5, which list(s) would you recommend to him ?

In [87]:
print("List Visit with Preferences 1, 2, 4 and 5:")
how_to_visit_Paris(1, 1, 0, 1, 1)

List Visit with Preferences 1, 2, 4 and 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
PC - La Place de la Concorde
TE - La Tour Eiffel
TM - la Tour Montparnasse
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


(l) If Mr. Doe wishes, at the same time, to take into account Preference 1, Preference 2, Preference 3, Preference 4 and
Preference 5, which list(s) would you recommend to him ?

In [88]:
print("List Visit with Preferences 1, 2, 3, 4 and 5:")
how_to_visit_Paris(1, 1, 1, 1, 1)

List Visit with Preferences 1, 2, 3, 4 and 5:
AC - l’Avenue des Champs-Elysées
AT - l’Arc de triomphe
CA - les Catacombes
PC - La Place de la Concorde
TE - La Tour Eiffel
TM - la Tour Montparnasse
Number of places in the ListVisit -  6.0

Are the obtained lists different from the solution ListVisit1 ?
Obtained list is different from the ListVisit1.


3. Let be :

• %Dur the ranking of the touristic sites obtained by observing only the Duration criterion (see the column “Duration” of the Table above)

• %App the ranking of the touristic sites obtained by observing only the Appreciations criterion (see the column “Appreciations” of the Table above)

• %Pri the ranking of the touristic sites obtained by observing only the Price criterion (see the column “Price” of the Table above)

Are these rankings two rankings different ? To answer this question, you can use the Kendall 1 or Spearman 2
rank correlation coefficient.

In [89]:
df = pd.DataFrame(data={'places': ["TE", "ML", "AT", "MO", "JT", "CA", "CP", "CN", "BS", "SC", "PC", "TM", "AC"],
                        'duration': [4.5, 3, 1, 2, 1.5, 2, 2.5, 2, 2, 1.5, 0.75, 2, 1.5],
                         'price': [15.5, 12, 9.5, 11, 0, 10, 10, 5, 8, 8.5, 0, 15, 0],
                         'rating': [5, 4, 3, 2, 3, 4, 1, 5, 4, 1, 3, 2, 5]})
df

,places,duration,price,rating
0,TE,4.50,15.5,5
1,ML,3.00,12.0,4
2,AT,1.00,9.5,3
3,MO,2.00,11.0,2
4,JT,1.50,0.0,3
5,CA,2.00,10.0,4
6,CP,2.50,10.0,1
7,CN,2.00,5.0,5
8,BS,2.00,8.0,4
9,SC,1.50,8.5,1


In [91]:
df['score_ranked_rating']=df['rating'].rank(ascending=0,method='dense')
df['score_ranked_duration']=df['duration'].rank(ascending=0,method='dense')
df['score_ranked_price']=df['price'].rank(ascending=1,method='dense')
df

,places,duration,price,rating,score_ranked_rating,score_ranked_duration,score_ranked_price
0,TE,4.50,15.5,5,1.0,1.0,10.0
1,ML,3.00,12.0,4,2.0,2.0,8.0
2,AT,1.00,9.5,3,3.0,6.0,5.0
3,MO,2.00,11.0,2,4.0,4.0,7.0
4,JT,1.50,0.0,3,3.0,5.0,1.0
5,CA,2.00,10.0,4,2.0,4.0,6.0
6,CP,2.50,10.0,1,5.0,3.0,6.0
7,CN,2.00,5.0,5,1.0,4.0,2.0
8,BS,2.00,8.0,4,2.0,4.0,3.0
9,SC,1.50,8.5,1,5.0,5.0,4.0


In [95]:
from scipy.stats import kendalltau

# calculate kendall's correlation
coef, p = kendalltau(df['score_ranked_rating'], df['score_ranked_duration'])
coef_1, p = kendalltau(df['score_ranked_price'], df['score_ranked_rating'])
coef_2, p = kendalltau(df['score_ranked_price'], df['score_ranked_duration'])

print('Kendall correlation coefficient on rating and duration: %.3f' % coef)
print('Kendall correlation coefficient on price and rating: %.3f' % coef_1)
print('Kendall correlation coefficient on price and duration: %.3f' % coef_2)

Kendall correlation coefficient on rating and duration: 0.152
Kendall correlation coefficient on price and rating: 0.085
Kendall correlation coefficient on price and duration: -0.606
